Load API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Load Contents from Brillar Bank

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls()['brillar_bank'])
data = loader.load()

In [3]:
from helpers import crawl

data = crawl("https://python.langchain.com/docs", max_depth=1)

Crawling: https://python.langchain.com/docs
Crawling: https://python.langchain.com/docs/how_to/ensemble_retriever/
Crawling: https://python.langchain.com/docs/how_to/output_parser_custom/
Crawling: https://python.langchain.com/docs/how_to/self_query/
Crawling: https://python.langchain.com/docs/how_to/query_no_queries/
Crawling: https://python.langchain.com/docs/how_to/contextual_compression/
Crawling: https://python.langchain.com/docs/concepts/
Crawling: https://python.langchain.com/docs/how_to/callbacks_runtime/
Crawling: https://python.langchain.com/docs/how_to/sql_query_checking/
Crawling: https://python.langchain.com/docs/how_to/tools_error/
Crawling: https://python.langchain.com/docs/how_to/dynamic_chain/
Crawling: https://python.langchain.com/docs/how_to/routing/
Crawling: https://python.langchain.com/docs/how_to/prompts_partial/
Crawling: https://python.langchain.com/docs/how_to/document_loader_custom/
Crawling: https://python.langchain.com/docs/tutorials/retrievers/
Crawling: h

Split the Loaded Data into Chunks

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Instantiate Elasticsearch Store

In [13]:
from langchain_elasticsearch import ElasticsearchStore, SparseVectorStrategy, BM25Strategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

sparse_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="brillar_bank",
    strategy=SparseVectorStrategy(model_id=".elser_model_2"),
        bulk_kwargs={
            "chunk_size": 500,
            "max_chunk_bytes": 15000000,
            "request_timeout": 3600,
        },
    
)

In [8]:
from langchain_elasticsearch import ElasticsearchStore, SparseVectorStrategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

sparse_vector_store = ElasticsearchStore(
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="brillar_bank",
    strategy=SparseVectorStrategy(model_id=".elser_model_2"),
)

Get LLM

In [15]:
from helpers import get_llm

llm = get_llm("gpt-4o")

Get Retriever

In [16]:
retriever = sparse_vector_store.as_retriever(search_kwargs={"k": 5})

Create Chain

In [17]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain, invoke_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [19]:
from helpers.test_data import get_questions

questions = get_questions()['brillar_bank']
for question in questions:
  result = invoke_conversational_retrieval_chain(chain, question)['answer']
  with open("./bbank_4o_test.txt", "+a") as file:
    file.write(f'Question: {question}\nAnswer: {result}\n\n')

d:\Code\langchain_python\virtualenv\Lib\site-packages\langchain_elasticsearch\vectorstores.py:764: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  hits = self._store.search(
d:\Code\langchain_python\virtualenv\Lib\site-packages\langchain_elasticsearch\vectorstores.py:764: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  hits = self._store.search(
d:\Code\langchain_python\virtualenv\Lib\site-packages\langchain_elasticsearch\vectorstores.py:764: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  hits = self._store.search(
d:\Code\langchain_python\virtualenv\Lib\site-packages\langchain_elasticsearch\vectorstores.py:764: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  hits = self._store.search(
d:\Code\langchain_python\virtualenv\Lib\site-packages\langchain_elasticsearch\vectorstores.py:764: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector ins